# Modified Transformer Architecture

Sources:

- [correct `transformer implementation` from scratch in `pytorch`](https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb) (in-depth tutorial from same author: [part1](https://towardsdatascience.com/all-you-need-to-know-about-attention-and-transformers-in-depth-understanding-part-1-552f0b41d021), [part2](https://towardsdatascience.com/all-you-need-to-know-about-attention-and-transformers-in-depth-understanding-part-2-bf2403804ada))
- [nice visuals for understanding `multi-head attention`](http://jalammar.github.io/illustrated-transformer/)
- [`positional encoding`](https://machinelearningmastery.com/a-gentle-introduction-to-positional-encoding-in-transformer-models-part-1/)
- [kaggle transformer code: ❗Contains mistakes (see comments), but nice overall explanation](https://www.kaggle.com/code/arunmohan003/transformer-from-scratch-using-pytorch)



In [235]:
!python --version

Python 3.9.13


In [236]:
import os
import math

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
seed = 42
torch.manual_seed(seed)
print(f"torch version: {torch.__version__}")

torch version: 2.4.1+cpu


# Dataset

For details about the dataset see `data_handling.ipynb`

In [237]:
SAVE_FOLDER = "processed_dataset"

df = pd.read_pickle(os.path.join(SAVE_FOLDER, "data.pkl"))
df.head()

,x_dominant_hand_0,x_dominant_hand_1,x_dominant_hand_2,x_dominant_hand_3,x_dominant_hand_4,x_dominant_hand_5,x_dominant_hand_6,x_dominant_hand_7,x_dominant_hand_8,x_dominant_hand_9,...,z_dominant_hand_11,z_dominant_hand_12,z_dominant_hand_13,z_dominant_hand_14,z_dominant_hand_15,z_dominant_hand_16,z_dominant_hand_17,z_dominant_hand_18,z_dominant_hand_19,z_dominant_hand_20
sequence_id,,,,,,,,,,,,,,,,,,,,,
1816796431,0.408832,0.519912,0.612159,0.707576,0.797313,0.494709,0.532817,0.553556,0.566219,0.391196,...,-0.245855,-0.269148,-0.129743,-0.251501,-0.278687,-0.266530,-0.152852,-0.257519,-0.275822,-0.266876
1816796431,0.398663,0.523662,0.638807,0.744236,0.832567,0.538486,0.564302,0.581011,0.597674,0.441541,...,-0.370770,-0.408097,-0.185217,-0.325494,-0.343373,-0.328294,-0.203126,-0.315719,-0.326104,-0.314282
1816796431,0.419290,0.509726,0.593165,0.685492,0.777913,0.483669,0.510993,0.536410,0.564583,0.393016,...,-0.285770,-0.318548,-0.155317,-0.274822,-0.312119,-0.316411,-0.181363,-0.286298,-0.316182,-0.322671
1816796431,0.398764,0.498118,0.583356,0.677779,0.775966,0.481279,0.491659,0.524974,0.571944,0.412262,...,-0.235725,-0.267054,-0.141380,-0.219369,-0.256553,-0.273690,-0.170996,-0.240285,-0.266193,-0.278110
1816796431,0.420213,0.495650,0.571790,0.659049,0.749740,0.485707,0.475930,0.501727,0.539150,0.438294,...,-0.186706,-0.217181,-0.107740,-0.165642,-0.201059,-0.222898,-0.131329,-0.183113,-0.208774,-0.225284


In [238]:
metadata_df = pd.read_csv(os.path.join(SAVE_FOLDER, "metadata.csv"), header=0)

max_phrase_len = max([len(it) for it in metadata_df.phrase.values])
possible_characters = sorted(set.union(*[set(p) for p in metadata_df.phrase.values]))
token_map = {c: i+3 for i, c in enumerate(possible_characters)}
token_map['P'] = 0 # padding
token_map['<'] = 1 # SOS
token_map['>'] = 2 # EOS
metadata_df.phrase = metadata_df.phrase.apply(lambda it: np.array([token_map[c] for c in '<'+it+'>'+('P'*(max_phrase_len-len(it)))], dtype=np.int32))

index = {row[0]: {"phrase": row[1], "signer_id": row[2]} for row in metadata_df.to_numpy()}
str(index)[:100] + "..."

"{1816796431: {'phrase': array([ 1,  7,  3, 16, 31, 18, 18, 24, 21, 28, 34, 32, 18,  2,  0,  0,  0,\n ..."

In [239]:
# from sklearn.model_selection import train_test_split

# sequence_ids = df.index.unique()

# train_ids, temp_ids = train_test_split(sequence_ids, test_size=0.3, random_state=seed)
# valid_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=seed)

# train_df = df[df.index.isin(train_ids)]
# valid_df = df[df.index.isin(valid_ids)]
# test_df = df[df.index.isin(test_ids)]

## Convolution-based Transformer Dataset

In [240]:
class TransformerDataset(torch.utils.data.Dataset):
    def __init__(self, df, meta_data, seq_len=256, padding_value=0.0):
        self.df = df
        self.meta_data = meta_data
        self.sequence_ids = df.index.unique()
        self.padding_value = padding_value
        self.seq_len = seq_len

    def __len__(self):
        return len(self.sequence_ids)

    def __getitem__(self, idx):
        sequence_id = self.sequence_ids[idx]
        x_values = torch.tensor(self.df.loc[sequence_id].values, dtype=torch.float32)

        x_mask = torch.ones(self.seq_len, dtype=torch.float32)

        # Apply padding if the sequence is shorter than seq_len
        if x_values.shape[0] < self.seq_len:
            x_mask[x_values.shape[0]:] = 0.0
            padding_size = self.seq_len - x_values.shape[0]
            padding = torch.full((padding_size, x_values.shape[1]), self.padding_value)
            x_values = torch.cat([x_values, padding], dim=0)
        elif x_values.shape[0] > self.seq_len:
            # Truncate the sequence if it's longer than seq_len
            x_values = x_values[:self.seq_len]

        y_phrase = self.meta_data[sequence_id]['phrase']
        return x_values, x_mask, y_phrase

dataset = TransformerDataset(df, index)

In [241]:
dataset.__getitem__(0)

(tensor([[ 0.4088,  0.5199,  0.6122,  ..., -0.2575, -0.2758, -0.2669],
         [ 0.3987,  0.5237,  0.6388,  ..., -0.3157, -0.3261, -0.3143],
         [ 0.4193,  0.5097,  0.5932,  ..., -0.2863, -0.3162, -0.3227],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [242]:
from torch.utils.data import DataLoader

# NOTE Refactor DataSet (move huge objects outside) before increasing number of workers
transformer_dataloader = DataLoader(dataset, batch_size=32, num_workers=0)

for it in transformer_dataloader:
    print("Shape of x:")
    print(it[0].shape)
    print("\nMask for input:")
    print(it[1].shape)
    print(it[1])
    print("\nTarget:")
    print(it[2].shape)
    print(it[2]) # NOTE dec inp: it[1][:, :-1], target: it[1][:, 1:]
    break

Shape of x:
torch.Size([32, 256, 63])

Mask for input:
torch.Size([32, 256])
tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]])

Target:
torch.Size([32, 33])
tensor([[ 1,  7,  3,  ...,  0,  0,  0],
        [ 1,  5,  7,  ...,  0,  0,  0],
        [ 1, 13, 12,  ...,  0,  0,  0],
        ...,
        [ 1,  8,  5,  ...,  0,  0,  0],
        [ 1,  8,  5,  ...,  0,  0,  0],
        [ 1, 26, 14,  ...,  0,  0,  0]], dtype=torch.int32)


In [246]:
%timeit next(iter(transformer_dataloader))

113 ms ± 1.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## GNN-based Transformer Dataset

In [452]:
# Based on: https://github.com/pyg-team/pytorch_geometric/blob/master/torch_geometric/loader/dataloader.py

from torch_geometric.data import Data, Batch

class SequentialGraphDataLoader:
    def __init__(self, dataset: TransformerDataset, batch_size: int = 32, shuffle: bool = False):
        # Wrap the TransformerDataset into default data loader
        self.transformer_dataloader = DataLoader(
            dataset=dataset, batch_size=batch_size, num_workers=0, shuffle=shuffle
        )

        # Precompute edge_index and "batch" for batch
        connections = [
            (0,1), (0,5), (0,17), (1,2), (2,3), (3,4),
            (5,6), (5,9), (6,7), (7,8), (9,10), (9,13),
            (10,11), (11,12), (13,14), (13,17), (14,15),
            (15,16), (17,18), (18,19), (19,20)
        ]
        edges = []
        for a, b in connections:
            edges.append([a, b])
            edges.append([b, a])  # Add the reverse connection
        self.edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

        # NOTE Edge index calculation can be done without using Batch.from_data_list(...) -> improve performance
        # For now this is good enough for a one time run
        example_batch = next(iter(self.transformer_dataloader))
        seq_len = len(example_batch[0][0])
        example_graph_features = torch.zeros(21, 3)
        data_list = [Data(x=example_graph_features, edge_index=self.edge_index) for _ in range(batch_size*seq_len)]
        self.mini_batch = Batch.from_data_list(data_list)
        self.edge_index = self.mini_batch.edge_index
        self.batch = self.mini_batch.batch

        self.iterator = iter(self.transformer_dataloader)

    def __iter__(self):
        self.iterator = iter(self.transformer_dataloader)
        return self

    def __next__(self):
        data = next(self.iterator)
        return [*data, self.edge_index, self.batch]
    
    def __len__(self):
        return len(self.transformer_dataloader) # NOTE how many batches

In [443]:
graph_dataloader = SequentialGraphDataLoader(dataset, batch_size=32)

# for it in graph_dataloader:
#     break

%timeit next(iter(graph_dataloader))

122 ms ± 2.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Constructing the Transformer

## General Components

In [284]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [285]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_len, n=10000):
        super(PositionalEncoding, self).__init__()

        assert d_model % 2 == 0, "d_model must be even"
        
        pe = torch.zeros(max_seq_len, d_model)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(n) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # NOTE
        # register buffer in Pytorch ->
        # If you have parameters in your model, which should be saved and restored in the state_dict,
        # but not trained by the optimizer, you should register them as buffers.
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        # Broadcasting mechanism (automatically works for multiple batches even when shapes don't match along batch dim)
        return x + self.pe[:, :x.size(1)]

In [444]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

## Encoder Embeddings

In [287]:
class PositionalLandmarkEmbedding(nn.Module):
    def __init__(self, max_seq_len, d_model, num_features, num_conv_layers, filter_size):
        super(PositionalLandmarkEmbedding, self).__init__()

        self.d_model = d_model

        first_conv = nn.Conv1d(in_channels=num_features, out_channels=d_model, kernel_size=filter_size, padding='same')
        rest_of_convs = [
            nn.Conv1d(in_channels=d_model, out_channels=d_model, kernel_size=filter_size, padding='same')
            for _ in range(num_conv_layers-1)
        ]
        
        self.conv_block = nn.Sequential(
            first_conv, *rest_of_convs
        )
        
        self.pos_encoding = PositionalEncoding(d_model, max_seq_len)

    def forward(self, x):
        # x is expected to be of shape (batch_size, seq_len, num_of_features)
        x = x.permute(0, 2, 1) # (batch_size, num_of_features, seq_len)
        x = self.conv_block(x)
        # TODO experiment with scaling the output
        # x *= math.sqrt(self.d_model)
        x = x.permute(0, 2, 1) # (batch_size, seq_len, d_model)
        x = self.pos_encoding(x)
        return x

In [291]:
ple = PositionalLandmarkEmbedding(256, 128, 63, 3, 11)
x = next(iter(transformer_dataloader))[0]
%timeit ple(x)

52.9 ms ± 6.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [445]:
from torch_geometric.nn import GCNConv, GATConv, GATv2Conv
from torch_geometric.nn import global_mean_pool

class PositionalGraphEmbedding(nn.Module):
    def __init__(self, max_seq_len, d_model, num_features_per_node):
        super(PositionalGraphEmbedding, self).__init__()
        self.d_model = d_model
        self.num_features_per_node = num_features_per_node

        # TODO experiment with hyperparameters
        hidden_dim = 256
        self.conv1 = GATConv(num_features_per_node, hidden_dim, heads=1, concat=False)
        self.conv2 = GATConv(hidden_dim, d_model, heads=1, concat=False)
        
        self.pos_encoding = PositionalEncoding(d_model, max_seq_len)

    def forward(self, x, edge_index, batch):
        # x ~ (batch_size, sequence_len, num_features)
        batch_size = x.shape[0]
        seq_len = x.shape[1]

        x = x.reshape(-1, self.num_features_per_node) # (batch_size*sequence_len*num_nodes, num_features_per_node)

        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # (batch_size*sequence_len, d_model)

        x = x.reshape(batch_size, seq_len, self.d_model)
        x = self.pos_encoding(x)
        return x

In [ ]:
x, x_mask, y, edge_index, batch = next(iter(graph_dataloader))
layer = PositionalGraphEmbedding(256, 128, 3)

# NOTE
# Running the Graph-based layers on each graph separately is more than 10x slower than running on the mini batch
# To make it faster:
# - run on GPU
# - rewrite GNN layers using the fact that all graphs will have the same structure
%timeit layer(x, edge_index, batch)

## Rest of the Transformer

In [43]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [44]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.masked_self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.masked_self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [453]:
class Transformer(nn.Module):
    def __init__(self, num_inp_features, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        # self.encoder_embedding = PositionalLandmarkEmbedding(
        #     max_seq_len=max_seq_length,
        #     d_model=d_model,
        #     num_features=num_inp_features,
        #     num_conv_layers=3, # TODO experiment with this!
        #     filter_size=11 # TODO experiment with this!
        # )
        self.encoder_embedding = PositionalGraphEmbedding(
            max_seq_len=max_seq_length,
            d_model=d_model,
            num_features_per_node=3
        )

        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, x_mask, tgt, edge_index=None, batch=None):
        src_mask, tgt_mask = self.generate_mask(x_mask, tgt)
        src_embedded = self.dropout(self.encoder_embedding(src, edge_index, batch))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

# Training

In [454]:
tgt_vocab_size=len(token_map)

transformer = Transformer(
    num_inp_features=63,
    tgt_vocab_size=tgt_vocab_size,
    d_model=128,
    num_heads=8,
    num_layers=2,
    d_ff=64,
    max_seq_length=max(max_phrase_len+2, 256),
    dropout=0.1
)

In [449]:
# # Optional lr scheduling as in Attention Is All You Need
# class LinearWarmupInverseSquarerootDecay(torch.optim.lr_scheduler.LambdaLR):
#     def __init__(self, d_model, warmup_steps=4000, optimizer=None):
#         self.d_model = d_model
#         self.warmup_steps = warmup_steps
#         super(LinearWarmupInverseSquarerootDecay, self).__init__(optimizer, self.lr_lambda)

#     def lr_lambda(self, step_num):
#         return (self.d_model ** -0.5) * min(step_num ** -0.5 if step_num != 0 else 1e20, step_num * (self.warmup_steps ** -1.5))
    
# steps = np.arange(0, 10000)
# learning_rates = [scheduler.lr_lambda(step) for step in steps]

# plt.figure(figsize=(6, 3))
# plt.plot(steps, learning_rates, label='Learning Rate')
# plt.title('Learning Rate Schedule')
# plt.xlabel('Step Number')
# plt.ylabel('Learning Rate')
# plt.grid(True)
# plt.legend()
# plt.show()

# scheduler = LinearWarmupInverseSquarerootDecay(d_model=d_model, optimizer=optimizer)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=0) # ignoring padding
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9, weight_decay=0.0)
# optimizer = torch.optim.Adam(transformer.parameters())

transformer.train()

for epoch in range(1):
    epoch_loss = 0
    total_correct = 0
    total_tokens = 0

    for batch in graph_dataloader:
        # src_data, x_mask, tgt_data = batch
        src_data, x_mask, tgt_data, edge_index, batch_ = batch
        optimizer.zero_grad()
        output = transformer(src_data, x_mask, tgt_data[:, :-1])

        # Calculate loss
        loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1).long())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        # Calculate accuracy
        output_tokens = output.argmax(dim=-1)  # Shape: (batch_size, seq_len)
        non_pad_mask = tgt_data[:, 1:] != 0  # Ignore padding tokens (mask for target data)
        correct = (output_tokens == tgt_data[:, 1:]) & non_pad_mask  # Compare predictions to targets and ignore padding
        total_correct += correct.sum().item()
        total_tokens += non_pad_mask.sum().item()

    avg_loss = epoch_loss / len(graph_dataloader)
    accuracy = total_correct / total_tokens if total_tokens > 0 else 0
    print(f"Epoch: {epoch+1}, Average Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

# TODO
# Masked Accuracy (+other metrics)
# Shuffle dataset not only in the batch
# read extra stuff in readme